In [ ]:
import gurobipy as gb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import function_library_assignment_1 as fnc

plt.rcParams['font.size']=12
plt.rcParams['font.family']='serif'
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False  
plt.rcParams['axes.spines.bottom'] = True     
plt.rcParams["axes.grid"] =True
plt.rcParams['grid.linestyle'] = '-.' 
plt.rcParams['grid.linewidth'] = 0.4

In [ ]:
n_bus = 24
S_base_3ph = 100
gen_data = fnc.read_data('gen_data')
system_demand = fnc.read_data('system_demand')
load_distribution = fnc.read_data('load_distribution')
gen_data = fnc.read_data('gen_data')
gen_costs = fnc.read_data('gen_costs')
line_data = fnc.read_data('line_data')
branch_matrix = fnc.read_data('branch_matrix')
wind_data = fnc.read_data('wind_data')

In [ ]:
gens_map, wf_map = fnc.mapping_dictionaries(gen_data)

gens_map.get(14) #example: get the generator indices at bus 14 (15 in the assignment formulation)

[4, 5]

# Task 4 - BESS Optimized Multi-Step OPF

In [ ]:
#for 3 batteries in total
n_hours = 24
# Define input parameters of batteries

SOC_max = 450 / S_base_3ph # Maximum SOC capacity 
SOC_ini = 0.25 * SOC_max # initial SOC at start
p_BESS = 150 / S_base_3ph # Dis-/charging capacity
eta_chg = 0.9 # charging efficiency
eta_dischg = 1.1 # discharging efficiency

# Define the parameters for BESS placement
n_bess = 3  # Number of available BESS units
nodes = range(n_bus)  # List of nodes in the power system
bess_capacity = [4.5, 9, 13.5]  # BESS capacity options at each node
max_capacity = 13.5 #maxium BESS capacity at each node

       
#def solve_mp_multihour(BESS: bool = True, print_variable_results: bool = True, show_plot: bool = False, save_fig: bool = False): #'NO' by default and 'YES' if BESS is included 
    
# Define the Gurobi model
mp = gb.Model('Integrated_BESS_Optimization')

hourly_loads = {} #dictionary containing the hourly load distributions

for t in range(n_hours):
    load = np.zeros(n_bus)
    demand = system_demand['System Demand'][t]  # Update demand for each hour

    #Saving the load for each bus in a numpy array accounting for the system load destribution
    for n in load_distribution['Node'].unique():
        load[n] = load_distribution.loc[load_distribution['Node'] == n, r'% of system load'] / 100 * demand / S_base_3ph #per-unitized load

    hourly_loads[t] =  load # Reset load for each hour

# Add variables for each hour
p_G = mp.addVars(len(gen_data.index), n_hours, lb=0, ub=gb.GRB.INFINITY, name="P_G")
p_W = mp.addVars(len(wind_data.columns), n_hours, lb=0, ub=gb.GRB.INFINITY, name="P_W")
theta = mp.addVars(n_bus, n_hours, lb=-gb.GRB.INFINITY, ub=gb.GRB.INFINITY, name="theta")

# Add variables to the Gurobi model
bess_placement = mp.addVars(n_bus, n_bess, vtype=gb.GRB.BINARY, name="BESS_Placement") #binary variable for (no) BESS placement at nodes
soc_total = mp.addVars(n_bus, n_bess, n_hours, lb=0, ub=max_capacity, name="SOC") #maximum combined state of charge
soc = mp.addVars(n_bus, n_bess, n_hours, lb=0, ub= SOC_max, name="BESS_SOC") #state of charge for each battery in every hour
charge = mp.addVars(n_bus, n_bess, n_hours, lb=0, ub= 1.5 , name="BESS_P_ch") #charging power for each battery in every hour
discharge = mp.addVars(n_bus, n_bess, n_hours, lb=0, ub= 1.5, name="BESS_P_disch") #discharging power for each battery in every hour

# Set objective function for each hour
obj = gb.quicksum(gen_costs['C (DKK/MWh)'][k] * p_G[k,t] * S_base_3ph for k in range(len(gen_costs.index)) for t in range(n_hours))
mp.setObjective(obj, gb.GRB.MINIMIZE)

#Managing line capacities - remembering that the "to" and "from" are not zero-indexed in the data
for n in nodes:
    for k in range(n, n_bus): #Avoid duplicates by starting the indexing of k at i
            if (n != k) and (branch_matrix[n,k] != 0):
                mp.addConstrs((theta[n,t] - theta[k,t]) * branch_matrix[n,k] <= (line_data.loc[(line_data['From'] == n + 1) & (line_data['To'] == k + 1), 'Capacity pu'].sum()) for t in range(n_hours))
                mp.addConstrs((theta[n,t] - theta[k,t]) * branch_matrix[n,k] >= -1 * (line_data.loc[(line_data['From'] == n + 1) & (line_data['To'] == k + 1), 'Capacity pu'].sum()) for t in range(n_hours))

#Power balance equation (without BESS)
# for n in nodes:
#     mp.addConstrs(gb.quicksum(p_G[g,t] for g in gens_map.get(n)) + gb.quicksum(p_W[w,t] for w in wf_map.get(n)) - hourly_loads.get(t)[n] == 
#                 theta[n,t] * branch_matrix[n,n] + gb.quicksum(theta[k,t] * branch_matrix[n,k] for k in range(n_bus) if k != n) for t in range(n_hours))

# Power balance equation (with BESS)
for n in nodes:
    for t in range(n_hours):
        if bess_placement == 1:
            expr = (gb.quicksum(p_G[g, t] for g in gens_map.get(n)) + gb.quicksum(p_W[w, t] for w in wf_map.get(n)) + gb.quicksum(discharge[n,b,t] - charge[n,b,t] for b in range(n_bess)) - hourly_loads.get(t)[n] == 
                        theta[n,t] * branch_matrix[n,n] + gb.quicksum(theta[k,t] * branch_matrix[n,k] for k in range(n_bus) if k != n) for t in range(n_hours))
        else:
            expr=(gb.quicksum(p_G[g,t] for g in gens_map.get(n)) + gb.quicksum(p_W[w,t] for w in wf_map.get(n)) - hourly_loads.get(t)[n] == 
                theta[n,t] * branch_matrix[n,n] + gb.quicksum(theta[k,t] * branch_matrix[n,k] for k in range(n_bus) if k != n) for t in range(n_hours))
        
        mp.addConstrs(expr)


# BESS Constraints
# Constraints for BESS charging and discharging
for t in range(1, n_hours):  # Start from t=1, as the first time step is already defined
    for n in nodes:
        for b in range(n_bess):
            mp.addConstr(charge[n, b, t] <= (1 - bess_placement[n, b]) * 1.5)  # Charging only if BESS is placed
            mp.addConstr(discharge[n, b, t] <= bess_placement[n, b] * 1.5)  # Discharging only if BESS is placed

# Constraint to place exactly 3 BESS units
mp.addConstr(gb.quicksum(bess_placement[n,b] for n in nodes for b in range(n_bess)) == 3)

# Total capacity constraint for all BESS units
mp.addConstr(gb.quicksum(bess_capacity[b] * bess_placement[n,b] for n in nodes for b in range(n_bess)) <= max_capacity)

# State of charge constraints for each BESS unit at each node
for n in nodes:
    for b in range(n_bess):
        mp.addConstrs(
            soc[n, b, t] == soc[n, b, t - 1] + eta_chg * charge[n, b, t - 1] - eta_dischg * discharge[n, b, t - 1]
            for t in range(1, n_hours)
        )
        mp.addConstr(soc[n, b, 0] == SOC_ini)
        mp.addConstr(soc[n, b, n_hours - 1] >= SOC_ini)


#Power Constraints
mp.addConstrs(p_G[g,t] <= (gen_data['P max MW'].iloc[g] / S_base_3ph) for g in range(len(gen_data)) for t in range(n_hours)) #if the generator is on (s_G[i] = 1), then the limit is p_max
mp.addConstrs(p_G[g,t] >= 0 for g in range(len(gen_data)) for t in range(n_hours)) #P_min from the system should be disregarded to avoid having a mixed integer program
mp.addConstrs(p_W[w,t] <= wind_data.iloc[t, w] / S_base_3ph for w in range(len(wind_data.columns)) for t in range(n_hours)) #Maximum wind power is the per-unitized output of the non-curtailed wind farm in the hour t
mp.addConstrs(p_W[w,t] >= 0 for w in range(len(wind_data.columns)) for t in range(n_hours)) #Wind farms can be curtailed
mp.addConstrs(theta[0,t] == 0 for t in range(n_hours))           


# Optimize the model for each hour
mp.optimize()

# Display the optimal BESS placement
if mp.status == gb.GRB.OPTIMAL:
    print("Hourly Optimal Generation Results:")
    for t in range(n_hours):
        print(f"Hour {t + 1}:")
        for g in range(len(gen_data.index)):
            generation = p_G[g, t].X*S_base_3ph  
            print(f"Generator {g + 1}: {generation:.2f} MWh")

    print("\nHourly Wind Generation Results:")
    for t in range(n_hours):
        print(f"Hour {t + 1}:")
        for w in range(len(wind_data.columns)):
            wind_generation = p_W[w, t].X*S_base_3ph
            print(f"Wind Farm {w + 1}: {wind_generation:.2f} MWh")

    optimal_placement = {n: next((b for b in range(n_bess) if bess_placement[n, b].X > 0), None) for n in nodes}
    print("Optimal BESS Placement:")
    for n, placement in optimal_placement.items():
        if placement is not None:
            node_capacity = bess_capacity[placement] * S_base_3ph
            print(f"Node {n}: {node_capacity} MWh BESS")
        else:
            print(f"Node {n}: No BESS placed")        
else:
    print("Optimization did not converge to an optimal solution.")

if mp.status == gb.GRB.OPTIMAL:
    results = {} #to be included: branch flows, theta, generator outputs, (battery operation)
    branch_dict = {}
    theta_dict = {}
    gen_dict = {}
    wind_dict = {}
    bess_dict = {}




Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i5-6300U CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 21458 rows, 7992 columns and 81048 nonzeros
Model fingerprint: 0x9ab205f5
Variable types: 7920 continuous, 72 integer (72 binary)
Coefficient statistics:
  Matrix range     [9e-01, 2e+02]
  Objective range  [4e+03, 2e+04]
  Bounds range     [1e+00, 1e+01]
  RHS range        [3e-01, 1e+01]
Presolve removed 21391 rows and 7901 columns
Presolve time: 0.20s
Presolved: 67 rows, 91 columns, 200 nonzeros
Variable types: 67 continuous, 24 integer (24 binary)
Found heuristic solution: objective 1735452.6555

Explored 0 nodes (0 simplex iterations) in 0.25 seconds (0.09 work units)
Thread count was 4 (of 4 available processors)

Solution count 1: 1.73545e+06 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.735452655541e+06, best bound 1.7354526

In [ ]:
#max 3 batteries per node
#only get results for BESS if both power balance equation without BESS and power constraints are removed, 
#model won't run with power constraints but without eq
n_hours = 24
# Define input parameters of batteries

SOC_max = 450 / S_base_3ph # Maximum SOC capacity 
SOC_ini = 200 / S_base_3ph # initial SOC at start
p_BESS = 150 / S_base_3ph # Dis-/charging capacity
eta_chg = 0.9 # charging efficiency
eta_dischg = 1.1 # discharging efficiency

# Define the parameters for BESS placement
n_bess = 3  # Number of available BESS units
nodes = range(n_bus)  # List of nodes in the power system
bess_capacity = [4.5, 9, 13.5]  # BESS capacity options at each node
max_capacity = 13.5 #maxium BESS capacity at each node

       
#def solve_mp_multihour(BESS: bool = True, print_variable_results: bool = True, show_plot: bool = False, save_fig: bool = False): #'NO' by default and 'YES' if BESS is included 
    
# Define the Gurobi model
mp = gb.Model('Integrated_BESS_Optimization')

#mp.setParam('TimeLimit', 100)

hourly_loads = {} #dictionary containing the hourly load distributions

for t in range(n_hours):
    load = np.zeros(n_bus)
    demand = system_demand['System Demand'][t]  # Update demand for each hour

    #Saving the load for each bus in a numpy array accounting for the system load destribution
    for n in load_distribution['Node'].unique():
        load[n] = load_distribution.loc[load_distribution['Node'] == n, r'% of system load'] / 100 * demand / S_base_3ph #per-unitized load

    hourly_loads[t] =  load # Reset load for each hour

# Add variables for each hour
p_G = mp.addVars(len(gen_data.index), n_hours, lb=0, ub=gb.GRB.INFINITY, name="P_G")
p_W = mp.addVars(len(wind_data.columns), n_hours, lb=0, ub=gb.GRB.INFINITY, name="P_W")
theta = mp.addVars(n_bus, n_hours, lb=-gb.GRB.INFINITY, ub=gb.GRB.INFINITY, name="theta")
# Define 
s_G = mp.addVars(len(gen_data.index), n_hours, vtype=gb.GRB.BINARY, name="Generator_Status")#binary variables for generator status (on/off) (s_G)
s_W = mp.addVars(len(wind_data.columns), n_hours, vtype=gb.GRB.BINARY, name="WindFarm_Status")# binary variables for wind farm status (on/off) (s_W)


# Add variables to the Gurobi model
bess_placement = mp.addVars(n_bus, n_bess, vtype=gb.GRB.BINARY, name="BESS_Placement") #binary variable for (no) BESS placement at nodes
soc_total = mp.addVars(n_bus, n_bess, n_hours, lb=0, ub=max_capacity, name="SOC") #maximum combined state of charge
soc = mp.addVars(n_bus, n_bess, n_hours, lb=0, ub= SOC_max, name="BESS_SOC") #state of charge for each battery in every hour
charge = mp.addVars(n_bus, n_bess, n_hours, lb=0, ub= 1.5 , name="BESS_P_ch") #charging power for each battery in every hour
discharge = mp.addVars(n_bus, n_bess, n_hours, lb=0, ub= 1.5, name="BESS_P_disch") #discharging power for each battery in every hour

# Set objective function for each hour
obj = gb.quicksum(gen_costs['C (DKK/MWh)'][k] * p_G[k,t] * S_base_3ph for k in range(len(gen_costs.index)) for t in range(n_hours))
mp.setObjective(obj, gb.GRB.MINIMIZE)

#Managing line capacities - remembering that the "to" and "from" are not zero-indexed in the data
for n in nodes:
    for k in range(n, n_bus): #Avoid duplicates by starting the indexing of k at i
            if (n != k) and (branch_matrix[n,k] != 0):
                mp.addConstrs((theta[n,t] - theta[k,t]) * branch_matrix[n,k] <= (line_data.loc[(line_data['From'] == n + 1) & (line_data['To'] == k + 1), 'Capacity pu'].sum()) for t in range(n_hours))
                mp.addConstrs((theta[n,t] - theta[k,t]) * branch_matrix[n,k] >= -1 * (line_data.loc[(line_data['From'] == n + 1) & (line_data['To'] == k + 1), 'Capacity pu'].sum()) for t in range(n_hours))

#Power balance equation (without BESS)
# for n in nodes:
#     mp.addConstrs(gb.quicksum(p_G[g,t] for g in gens_map.get(n)) + gb.quicksum(p_W[w,t] for w in wf_map.get(n)) - hourly_loads.get(t)[n] == 
#                 theta[n,t] * branch_matrix[n,n] + gb.quicksum(theta[k,t] * branch_matrix[n,k] for k in range(n_bus) if k != n) for t in range(n_hours))

# Power balance equation (with/out BESS)
# Power balance equation (with BESS)
# for n in nodes:
#     for t in range(n_hours):
#         if bess_placement == 1:
#             expr = (gb.quicksum(p_G[g, t] for g in gens_map.get(n)) + gb.quicksum(p_W[w, t] for w in wf_map.get(n)) + gb.quicksum(discharge[n,b,t] - charge[n,b,t] for b in range(n_bess)) - hourly_loads.get(t)[n] == 
#                         theta[n,t] * branch_matrix[n,n] + gb.quicksum(theta[k,t] * branch_matrix[n,k] for k in range(n_bus) if k != n) for t in range(n_hours))
#         else:
#             expr=(gb.quicksum(p_G[g,t] for g in gens_map.get(n)) + gb.quicksum(p_W[w,t] for w in wf_map.get(n)) - hourly_loads.get(t)[n] == 
#                 theta[n,t] * branch_matrix[n,n] + gb.quicksum(theta[k,t] * branch_matrix[n,k] for k in range(n_bus) if k != n) for t in range(n_hours))
        
#         mp.addConstrs(expr)


for n in nodes:
        mp.addConstrs(gb.quicksum(p_G[g, t] for g in gens_map.get(n)) + gb.quicksum(p_W[w, t] for w in wf_map.get(n)) + gb.quicksum(discharge[n,b,t] - charge[n,b,t] for b in range(n_bess)) - hourly_loads.get(t)[n] == 
                        theta[n,t] * branch_matrix[n,n] + gb.quicksum(theta[k,t] * branch_matrix[n,k] for k in range(n_bus) if k != n) for t in range(n_hours))
        

# BESS Constraints
# Constraints for BESS charging and discharging
for t in range(1, n_hours):  # Start from t=1, as the first time step is already defined
    for n in nodes:
        for b in range(n_bess):
            mp.addConstr(charge[n, b, t] <= (1 - bess_placement[n, b]) * 1.5)  # Charging only if BESS is placed
            mp.addConstr(discharge[n, b, t] <= bess_placement[n, b] * 1.5)  # Discharging only if BESS is placed


# Each node can have 0, 1, 2, or 3 BESS units
mp.addConstrs(gb.quicksum(bess_placement[n, b] for b in range(n_bess)) >= 0 for n in nodes)
mp.addConstrs(gb.quicksum(bess_placement[n, b] for b in range(n_bess)) <= 3 for n in nodes)

# Capacity constraints: Ensure the total capacity at each node does not exceed the maximum
mp.addConstrs(gb.quicksum(bess_placement[n, b] * bess_capacity[b]  for b in range(n_bess)) <= max_capacity for n in nodes)

# State of charge constraints for each BESS unit at each node
for n in nodes:
    for b in range(n_bess):
        mp.addConstrs(
            soc[n, b, t] == soc[n, b, t - 1] + eta_chg * charge[n, b, t - 1] - eta_dischg * discharge[n, b, t - 1]
            for t in range(1, n_hours)
        )
        mp.addConstr(soc[n, b, 0] == SOC_ini)
        mp.addConstr(soc[n, b, n_hours - 1] >= SOC_ini)


#Power Constraints
mp.addConstrs(p_G[g,t] <= (gen_data['P max MW'].iloc[g] / S_base_3ph) for g in range(len(gen_data)) for t in range(n_hours)) #if the generator is on (s_G[i] = 1), then the limit is p_max
#mp.addConstrs(p_G[g,t] <= (gen_data['P max MW'].iloc[g] / S_base_3ph) * s_G[g,t] for g in range(len(gen_data)) for t in range(n_hours))
mp.addConstrs(p_G[g,t] >= 0 for g in range(len(gen_data)) for t in range(n_hours)) #P_min from the system should be disregarded to avoid having a mixed integer program
mp.addConstrs(p_W[w,t] <= wind_data.iloc[t, w] / S_base_3ph for w in range(len(wind_data.columns)) for t in range(n_hours)) #Maximum wind power is the per-unitized output of the non-curtailed wind farm in the hour t
#mp.addConstrs(p_W[w,t] <= wind_data.iloc[t, w] / S_base_3ph * s_W[w,t] for w in range(len(wind_data.columns)) for t in range(n_hours))
mp.addConstrs(p_W[w,t] >= 0 for w in range(len(wind_data.columns)) for t in range(n_hours)) #Wind farms can be curtailed
mp.addConstrs(theta[0,t] == 0 for t in range(n_hours)) 
        


# Optimize the model for each hour
mp.optimize()

# Display the optimal BESS placement
if mp.status == gb.GRB.OPTIMAL:
    print("Hourly Optimal Generation Results:")
    for t in range(n_hours):
        print(f"Hour {t + 1}:")
        for g in range(len(gen_data.index)):
            generation = p_G[g, t].X*S_base_3ph  
            print(f"Generator {g + 1}: {generation:.2f} MWh")

    print("\nHourly Wind Generation Results:")
    for t in range(n_hours):
        print(f"Hour {t + 1}:")
        for w in range(len(wind_data.columns)):
            wind_generation = p_W[w, t].X*S_base_3ph
            print(f"Wind Farm {w + 1}: {wind_generation:.2f} MWh")

    optimal_placement = {n: next((b for b in range(n_bess) if bess_placement[n, b].X > 0), None) for n in nodes}
    print("Optimal BESS Placement:")
    for t in range(n_hours):
        print(f"Hour {t + 1}:")
        for n, placement in optimal_placement.items():
            if placement is not None:
                node_capacity = bess_capacity[placement] * S_base_3ph
                print(f"Node {n}: {node_capacity} MWh BESS")
            else:
                print(f"Node {n}: No BESS placed")        
else:
    print("Optimization did not converge to an optimal solution.")


#Print solutions
if mp.status == gb.GRB.OPTIMAL:
    results = {} #to be included: branch flows, theta, generator outputs, (battery operation)
    branch_dict = {}
    theta_dict = {}
    gen_dict = {}
    wind_dict = {}
    bess_dict = {}
    


Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i5-6300U CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 8280 rows, 8424 columns and 23856 nonzeros
Model fingerprint: 0xea5a59fa
Variable types: 7920 continuous, 504 integer (504 binary)
Coefficient statistics:
  Matrix range     [9e-01, 2e+02]
  Objective range  [4e+03, 2e+04]
  Bounds range     [1e+00, 1e+01]
  RHS range        [3e-01, 1e+01]
Presolve removed 1296 rows and 2328 columns
Presolve time: 0.10s
Presolved: 6984 rows, 6096 columns, 22080 nonzeros
Variable types: 6024 continuous, 72 integer (72 binary)
Found heuristic solution: objective 1735452.6555

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   12328    1.4927616e+06   1.336886e+04   0.000000e+00      5s
   14298    1.5534214e+06   0.000000e+00   0.000000e+00      5s

Root relaxation: objective 1.55

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "c:\Users\Admin\anaconda3\Lib\site-packages\ipykernel\iostream.py", line 526, in write
    def write(self, string: str) -> Optional[int]:  # type:ignore[override]

KeyboardInterrupt: 


 15125  9862 1582100.81   49   18 1587189.09 1582062.04  0.32%   115 32230s
 15157  9895 1582323.67   57   13 1587189.09 1582062.04  0.32%   115 32235s
